# Hawk-Dove Evolutionarily Stable Strategy (ESS) Model

## Introduction

This notebook explores the classic **Hawk-Dove game**, a fundamental model in evolutionary game theory that explains how different behavioral strategies can coexist in a population.

### Key Concepts:

- **Hawk Strategy**: Aggressive behavior, fights for resources, risks injury
- **Dove Strategy**: Passive behavior, shares or retreats, avoids conflict
- **ESS (Evolutionarily Stable Strategy)**: A strategy that, once adopted by most of the population, cannot be invaded by a mutant strategy

### Why This Matters:

This model helps explain:
- Why aggressive and non-aggressive behaviors coexist in nature
- How evolution selects for optimal behavioral strategies
- The role of costs and benefits in evolutionary dynamics

Let's explore this interactively!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, FloatSlider, IntSlider
import pandas as pd

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## Understanding the Payoff Matrix

The Hawk-Dove game uses a **payoff matrix** to determine fitness gains/losses based on interactions:

| Player 2 \ Player 1 | Hawk | Dove |
|---------------------|------|------|
| **Hawk** | High risk/reward (fight) | Win (Hawk gets resource) |
| **Dove** | Lose (retreats) | Share (both get portion) |

### Parameters:
- **V (Value)**: Benefit of winning the resource
- **C (Cost)**: Cost of losing a fight (injury, energy, etc.)

The payoffs are calculated as follows:
- **Hawk vs Hawk**: (V - C) / 2 (50% chance to win, but risk injury)
- **Hawk vs Dove**: V (always wins against dove)
- **Dove vs Hawk**: 0 (retreats, gets nothing)
- **Dove vs Dove**: V / 2 (share resource equally)

## Core Calculation Functions

Let's define the key functions for calculating payoffs and finding equilibrium:

In [ ]:
def calculate_payoff_matrix(value_v, cost_c):
    """
    Calculate the payoff matrix for Hawk-Dove game.
    
    Parameters:
    -----------
    value_v : float
        Value of the resource
    cost_c : float
        Cost of losing a fight
    
    Returns:
    --------
    dict : Payoffs for each strategy pair
    """
    return {
        'hawk_hawk': (value_v - cost_c) / 2,
        'hawk_dove': value_v,
        'dove_hawk': 0,
        'dove_dove': value_v / 2
    }


def calculate_average_payoffs(hawk_share, payoffs):
    """
    Calculate average payoffs for each strategy given hawk population share.
    
    Parameters:
    -----------
    hawk_share : float
        Proportion of hawks in population (0-1)
    payoffs : dict
        Payoff matrix from calculate_payoff_matrix
    
    Returns:
    --------
    tuple : (avg_hawk_payoff, avg_dove_payoff)
    """
    dove_share = 1 - hawk_share
    
    avg_hawk_payoff = payoffs['hawk_hawk'] * hawk_share + payoffs['hawk_dove'] * dove_share
    avg_dove_payoff = payoffs['dove_hawk'] * hawk_share + payoffs['dove_dove'] * dove_share
    
    return avg_hawk_payoff, avg_dove_payoff


def find_equilibrium(payoffs, num_points=100000):
    """
    Find the ESS equilibrium point where hawk and dove payoffs are equal.
    
    Parameters:
    -----------
    payoffs : dict
        Payoff matrix
    num_points : int
        Resolution for finding equilibrium
    
    Returns:
    --------
    dict : Equilibrium information
    """
    hawk_share = np.linspace(0, 1, num_points)
    dove_share = 1 - hawk_share
    
    avg_hawk_payoff = payoffs['hawk_hawk'] * hawk_share + payoffs['hawk_dove'] * dove_share
    avg_dove_payoff = payoffs['dove_hawk'] * hawk_share + payoffs['dove_dove'] * dove_share
    
    payoff_diff = avg_hawk_payoff - avg_dove_payoff
    
    idx = np.where(np.diff(np.sign(payoff_diff)))[0]
    
    if len(idx) > 0:
        equilibrium_idx = idx[0]
        equilibrium_hawk_share = hawk_share[equilibrium_idx]
        equilibrium_payoff = (avg_hawk_payoff[equilibrium_idx] + avg_dove_payoff[equilibrium_idx]) / 2
        
        return {
            'found': True,
            'hawk_share': equilibrium_hawk_share,
            'payoff': equilibrium_payoff,
            'hawk_payoff': avg_hawk_payoff[equilibrium_idx],
            'dove_payoff': avg_dove_payoff[equilibrium_idx]
        }
    else:
        return {'found': False}

## Visualization Functions

In [ ]:
def plot_payoff_comparison(value_v, cost_c):
    """
    Plot average payoffs vs hawk population share.
    """
    payoffs = calculate_payoff_matrix(value_v, cost_c)
    equilibrium = find_equilibrium(payoffs)
    
    hawk_share = np.linspace(0, 1, 1000)
    avg_hawk_payoff = payoffs['hawk_hawk'] * hawk_share + payoffs['hawk_dove'] * (1 - hawk_share)
    avg_dove_payoff = payoffs['dove_hawk'] * hawk_share + payoffs['dove_dove'] * (1 - hawk_share)
    total_payoff = avg_hawk_payoff * hawk_share + avg_dove_payoff * (1 - hawk_share)
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    sns.lineplot(x=hawk_share, y=avg_hawk_payoff, ax=ax, label='Hawk Strategy', linewidth=2.5, color='#e74c3c')
    sns.lineplot(x=hawk_share, y=avg_dove_payoff, ax=ax, label='Dove Strategy', linewidth=2.5, color='#3498db')
    sns.lineplot(x=hawk_share, y=total_payoff, ax=ax, label='Population Average', linewidth=2.5, color='#27ae60', linestyle='--')
    
    ax.axhline(y=0, color='black', linewidth=0.8, alpha=0.5)
    ax.axvline(x=0.5, color='black', linewidth=0.8, linestyle=':', alpha=0.3)
    
    if equilibrium['found']:
        ax.axvline(x=equilibrium['hawk_share'], color='purple', linestyle='--', alpha=0.8, linewidth=2, 
                  label=f"ESS Equilibrium: {equilibrium['hawk_share']*100:.1f}% Hawks")
        ax.scatter([equilibrium['hawk_share']], [equilibrium['payoff']], color='purple', s=200, zorder=5)
    
    ax.set_xlabel('Proportion of Hawks in Population', fontsize=12, fontweight='bold')
    ax.set_ylabel('Average Payoff (Fitness)', fontsize=12, fontweight='bold')
    ax.set_title(f'Hawk-Dove ESS Model (V={value_v}, C={cost_c})', fontsize=14, fontweight='bold', pad=20)
    
    ax.legend(loc='best', fontsize=11, framealpha=0.95)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return payoffs, equilibrium


def print_analysis(value_v, cost_c, payoffs, equilibrium):
    """
    Print detailed analysis of the model.
    """
    print("="*60)
    print("PAYOFF MATRIX ANALYSIS")
    print("="*60)
    print(f"\nValue of resource (V): {value_v}")
    print(f"Cost of fighting (C): {cost_c}")
    print(f"\nPayoff for each strategy pair:")
    print(f"  Hawk vs Hawk: {payoffs['hawk_hawk']:.2f}")
    print(f"  Hawk vs Dove: {payoffs['hawk_dove']:.2f}")
    print(f"  Dove vs Hawk: {payoffs['dove_hawk']:.2f}")
    print(f"  Dove vs Dove: {payoffs['dove_dove']:.2f}")
    
    if equilibrium['found']:
        print(f"\n" + "="*60)
        print("EVOLUTIONARY STABLE STRATEGY (ESS)")
        print("="*60)
        print(f"\nEquilibrium found at: {equilibrium['hawk_share']*100:.2f}% Hawks")
        print(f"                  and: {(1-equilibrium['hawk_share'])*100:.2f}% Doves")
        print(f"\nAverage payoff at equilibrium: {equilibrium['payoff']:.2f}")
        print(f"  - Hawks get: {equilibrium['hawk_payoff']:.2f}")
        print(f"  - Doves get: {equilibrium['dove_payoff']:.2f}")
        print("\nInterpretation:")
        print(f"  → In a stable population, {equilibrium['hawk_share']*100:.1f}% of individuals")
        print(f"    will adopt the aggressive hawk strategy.")
        print(f"  → Both strategies have equal fitness at equilibrium.")
        print(f"  → Neither strategy can invade the other at this point.")
    else:
        print(f"\n" + "="*60)
        print("NO EQUILIBRIUM FOUND")
        print("="*60)
        print("\nWith these parameters, no stable equilibrium exists.")
        print("One strategy will dominate the population.")
    
    print("\n" + "="*60 + "\n")

## Time Series Simulation

Let's simulate how the population evolves over time using the **replicator equation**:

$$\frac{dx}{dt} = x \cdot (W_x - \bar{W})$$

Where:
- $x$ = proportion of hawks
- $W_x$ = fitness of hawks
- $\bar{W}$ = average fitness of population

In [ ]:
def simulate_population_dynamics(value_v, cost_c, initial_hawk_share, time_steps, dt=0.1):
    """
    Simulate population dynamics using the replicator equation.
    
    Parameters:
    -----------
    value_v, cost_c : float
        Model parameters
    initial_hawk_share : float
        Starting proportion of hawks (0-1)
    time_steps : int
        Number of time steps to simulate
    dt : float
        Time step size
    
    Returns:
    --------
    pd.DataFrame : Time series of population composition and payoffs
    """
    payoffs = calculate_payoff_matrix(value_v, cost_c)
    
    hawk_share = initial_hawk_share
    
    history = []
    
    for t in range(time_steps):
        dove_share = 1 - hawk_share
        
        avg_hawk_payoff = payoffs['hawk_hawk'] * hawk_share + payoffs['hawk_dove'] * dove_share
        avg_dove_payoff = payoffs['dove_hawk'] * hawk_share + payoffs['dove_dove'] * dove_share
        avg_population_payoff = avg_hawk_payoff * hawk_share + avg_dove_payoff * dove_share
        
        history.append({
            'time': t * dt,
            'hawk_share': hawk_share,
            'dove_share': dove_share,
            'hawk_payoff': avg_hawk_payoff,
            'dove_payoff': avg_dove_payoff,
            'avg_payoff': avg_population_payoff
        })
        
        dx = hawk_share * (avg_hawk_payoff - avg_population_payoff) * dt
        hawk_share = max(0, min(1, hawk_share + dx))
    
    return pd.DataFrame(history)


def plot_population_dynamics(value_v, cost_c, initial_hawk_share, time_steps=200, dt=0.1):
    """
    Plot population dynamics over time.
    """
    df = simulate_population_dynamics(value_v, cost_c, initial_hawk_share, time_steps, dt)
    equilibrium = find_equilibrium(calculate_payoff_matrix(value_v, cost_c))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    ax1.plot(df['time'], df['hawk_share'], label='Hawks', linewidth=2.5, color='#e74c3c')
    ax1.plot(df['time'], df['dove_share'], label='Doves', linewidth=2.5, color='#3498db')
    
    if equilibrium['found']:
        ax1.axhline(y=equilibrium['hawk_share'], color='purple', linestyle='--', linewidth=2, 
                   label=f"ESS: {equilibrium['hawk_share']*100:.1f}% Hawks")
    
    ax1.set_xlabel('Time', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Population Proportion', fontsize=12, fontweight='bold')
    ax1.set_title(f'Population Over Time\n(Starting: {initial_hawk_share*100:.1f}% Hawks)', 
                  fontsize=14, fontweight='bold')
    ax1.legend(fontsize=11, framealpha=0.95)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 1])
    
    ax2.plot(df['time'], df['hawk_payoff'], label='Hawk Fitness', linewidth=2.5, color='#e74c3c')
    ax2.plot(df['time'], df['dove_payoff'], label='Dove Fitness', linewidth=2.5, color='#3498db')
    ax2.plot(df['time'], df['avg_payoff'], label='Avg Fitness', linewidth=2.5, color='#27ae60', linestyle='--')
    
    ax2.set_xlabel('Time', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Fitness (Payoff)', fontsize=12, fontweight='bold')
    ax2.set_title('Fitness Over Time', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11, framealpha=0.95)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df

## 🎮 Interactive Exploration

Use the sliders below to explore how different parameters affect the ESS equilibrium. Try adjusting:

- **V (Value)**: What happens when the resource is more valuable?
- **C (Cost)**: How does fighting cost change the equilibrium?
- **Initial Hawks**: Does the starting population affect where it settles?

### Question to think about:
> What happens to the ESS when the cost of fighting (C) is very high compared to the value (V)?

In [ ]:
@interact(
    value_v=FloatSlider(value=50, min=10, max=100, step=5, description='Value (V):'),
    cost_c=FloatSlider(value=100, min=0, max=200, step=5, description='Cost (C):'),
    initial_hawk=FloatSlider(value=0.1, min=0.01, max=0.99, step=0.05, description='Initial Hawks:')
)
def explore_model(value_v, cost_c, initial_hawk):
    payoffs, equilibrium = plot_payoff_comparison(value_v, cost_c)
    print_analysis(value_v, cost_c, payoffs, equilibrium)
    df = plot_population_dynamics(value_v, cost_c, initial_hawk, time_steps=200, dt=0.1)
    
    return df

## 🔬 Experiment Guide

Try these experiments to deepen your understanding:

### Experiment 1: High Fighting Cost
- Set **Value (V) = 50** and **Cost (C) = 200**
- **What do you observe?** → Low hawk proportion in equilibrium
- **Why?** → Fighting is too costly, so dove strategy is favored

### Experiment 2: Low Fighting Cost
- Set **Value (V) = 100** and **Cost (C) = 50**
- **What do you observe?** → High hawk proportion
- **Why?** → Fighting is worth it when the risk is low and reward is high

### Experiment 3: Balanced Strategy
- Set **Value (V) = 50** and **Cost (C) = 100**
- **What do you observe?** → Mixed strategy (both hawks and doves coexist)
- **Why?** → When C > V, both strategies can survive together

### Experiment 4: Different Starting Points
- Keep V and C fixed, but try different **Initial Hawks** values
- **Does the equilibrium change?** → No! The ESS is stable
- **What does this tell us?** → Evolutionary dynamics converge to the same stable point


## 📚 Key Takeaways

### 1. Mixed ESS is Common
In nature, we often see a mix of aggressive and non-aggressive behaviors because:
- Pure strategies are vulnerable to invasion
- Mixed strategies provide stability

### 2. Cost-Benefit Ratio Matters
The equilibrium hawk proportion is: **p = V/C** (when C > V)
- Higher cost → fewer hawks
- Higher value → more hawks

### 3. Evolution Converges
- Different starting points lead to the same equilibrium
- The system is self-correcting
- Natural selection finds the optimal balance

### 4. Real-World Applications
- Animal territorial behavior
- Competition for resources
- Human conflict and cooperation
- Economics and game theory

## 🎓 Further Learning

### Extensions to Explore:
1. **Asymmetric Contests**: Different resource valuations for each player
2. **Multiple Strategies**: Add more behavioral strategies (e.g., "Retaliator")
3. **Spatial Structure**: Model individuals on a grid with local interactions
4. **Mutation Rates**: Add random mutations to study evolutionary stability

### References:
- Maynard Smith, J., & Price, G. R. (1973). The logic of animal conflict
- Maynard Smith, J. (1982). Evolution and the Theory of Games
- Hofbauer, J., & Sigmund, K. (1998). Evolutionary Games and Population Dynamics

---

**Happy exploring! 🧬**